In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Packages 

In [ ]:
import numpy as np
import pandas as pd
import time
import string
import re
import math

from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, TweedieRegressor,HuberRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error as mse


import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

import kerastuner as kt

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc, show, output_notebook
output_notebook()

import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
stop_words = stopwords.words('english')

import spacy
nlp = spacy.load('en_core_web_lg')
from spacy import displacy

import transformers
from transformers import BertTokenizer, TFBertModel, RobertaTokenizer, TFRobertaModel


In [ ]:
df_train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
df_test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
df_submission = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

print(" Training dataset shape : " + str(df_train.shape))
print(" Testing dataset shape : " + str(df_test.shape))

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
df_submission

In [ ]:
df_train['excerpt'][3]

## EDA

Lets start with null values 

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

From the above results we can say that only url_legal and license columns appear to be having missing values

### Target Distribution and std_error distribution

In [ ]:
# Train
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.distplot(df_train['target'], ax=ax, color ='green')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.distplot(df_train['standard_error'], ax=ax, color ='green')
plt.show()

### Lets Find Length of the excerpt column and Count of number of words in the excerpt column

In [ ]:
df_train['excerpt_len'] = df_train['excerpt'].apply(
    lambda x : len(x)
)
df_train['excerpt_word_count'] = df_train['excerpt'].apply(
    lambda x : len(x.split(' '))
)

In [ ]:
# Train
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.distplot(df_train['excerpt_len'], ax=ax, color ='green')
plt.show()

In [ ]:
# Train
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
sns.distplot(df_train['excerpt_word_count'], ax=ax, color ='green')
plt.show()

#### Vocabulary and its frequency before performing and data preprocessing 

In [ ]:
results = Counter()
df_train['excerpt'].str.lower().str.split().apply(results.update)
print(len(results.keys()))

In [ ]:
Word_Freq=df_train.excerpt.str.split(expand=True).stack().value_counts().reset_index()

In [ ]:
Word_Freq.to_csv('Train_Word_Freq.csv',index=False)

In [ ]:
Word_Freq_Test=df_test.excerpt.str.split(expand=True).stack().value_counts().reset_index()

In [ ]:
df_test

In [ ]:
Word_Freq_Test['index']

In [ ]:
common_Words=list(set(Word_Freq_Test['index']) & set(Word_Freq['index']))

In [ ]:
len(common_Words)

### Top N-Grams and Visualization

In [ ]:
def get_top_n_words(corpus, n = None):
    """
    A function that returns the top 'n' unigrams used in the corpus
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus) 
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()] 
    freq_sorted = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return freq_sorted[:n]

def get_top_n_bigram(corpus, n = None):
    """
    A function that returns the top 'n' bigrams used in the corpus
    """
    vec = CountVectorizer(ngram_range = (2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    freq_sorted = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return freq_sorted[:n]

def get_top_n_trigram(corpus, n = None):
    """
    A function that returns the top 'n' bigrams used in the corpus
    """
    vec = CountVectorizer(ngram_range = (3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    freq_sorted = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return freq_sorted[:n]

In [ ]:
top_unigram = get_top_n_words(df_train['excerpt'], 20)
words = [i[0] for i in top_unigram]
count = [i[1] for i in top_unigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#6baed6'] * 20))

p = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Unigrams", tools = "hover", tooltips = "@Word: @counts")
p.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
curdoc().theme = 'dark_minimal'
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.title.align = 'center'
p.xaxis.major_label_orientation = "vertical"
show(p)

In [ ]:
top_bigram = get_top_n_bigram(df_train['excerpt'], 20)
words = [i[0] for i in top_bigram]
count = [i[1] for i in top_bigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#a1dab4'] * 20))

p1 = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Bigrams", tools = "hover", tooltips = "@Word: @counts")
p1.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
# curdoc().theme = 'dark_minimal'
p1.xgrid.grid_line_color = None
p1.title.align = 'center'
p1.y_range.start = 0
p1.xaxis.major_label_orientation = "vertical"
show(p1)

In [ ]:
top_bigram = get_top_n_trigram(df_train['excerpt'], 20)
words = [i[0] for i in top_bigram]
count = [i[1] for i in top_bigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#a1dab4'] * 20))

p1 = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Bigrams", tools = "hover", tooltips = "@Word: @counts")
p1.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
# curdoc().theme = 'dark_minimal'
p1.xgrid.grid_line_color = None
p1.title.align = 'center'
p1.y_range.start = 0
p1.xaxis.major_label_orientation = "vertical"
show(p1)

We can see from the plots we have so many unwanted tokens. We need to remove them and clean the data

## Data Preprocessing

Data preprocessing is the process of converting raw data into a well-readable format to be used by a machine learning model.

### Data Cleaning
Here We will be doing list of things:
* Stop Word Removal: While doing Vectorization, we assign some number or we transform it into a vector, extremely common words like ‘the’, ‘and’, etc. will become very important features while they add little meaning to the text. Your model can often be improved if you don’t take those words into account. Stop words are just a list of words you don’t want to use as features. 
* Punctuation Removal
* URL/ Links Removal
* Removal of Numbers 
* Non- Ascii Characters removal
* Lemmatization

In [ ]:
def removeStopwords(text):
    doc = nlp(text)
    clean_text = ' '
    for txt in doc:
        if (txt.is_stop == False):
            clean_text = clean_text + " " + str(txt)        
    
    return clean_text

print("\033[1mText before removeStopwords function: \033[0m" + df_train['excerpt'][20])
print("\033[1mText after removeStopwords function: \033[0m" + removeStopwords(df_train['excerpt'][20]))

In [ ]:
def removePunctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

print("\033[1mText before removePunctuations function: \033[0m" + df_train['excerpt'][20])
print("\n")
print("\033[1mText after removePunctuations function: \033[0m" + removePunctuations(df_train['excerpt'][20]))

In [ ]:
def removeLinks(text):
    clean_text = re.sub('https?://\S+|www\.\S+', '', text)
    #https? will match both http and https
    #A|B, where A and B can be arbitrary REs, creates a regular expression that will match either A or B.
    #\S Matches any character which is not a whitespace character.
    #+ Causes the resulting RE to match 1 or more repetitions of the preceding RE. ab+ will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.
    return clean_text

In [ ]:
def removeNumbers(text):
    clean_text = re.sub(r'\d+', '', text)
    return clean_text

test_string = "CR 7"
(test_string,removeNumbers(test_string))

In [ ]:
def clean(text):
    text = text.lower() #Lets make it lowercase
    text = removeStopwords(text)
    text = removePunctuations(text)
    text = removeNumbers(text)
    text = removeLinks(text)
    return text

In [ ]:
df_train['excerpt_clean'] = df_train['excerpt'].apply(clean)
df_test['excerpt_clean'] = df_test['excerpt'].apply(clean)

df_train['excerpt_clean'] = df_train['excerpt_clean'].str.encode('ascii', 'ignore').str.decode('ascii')
df_test['excerpt_clean'] = df_test['excerpt_clean'].str.encode('ascii', 'ignore').str.decode('ascii')

df_train.head()

In [ ]:
df_train.head()

In [ ]:
results = Counter()
df_train['excerpt_clean'].str.lower().str.split().apply(results.update)
print(len(results.keys()))

In [ ]:
top_unigram = get_top_n_words(df_train['excerpt_clean'], 20)
words = [i[0] for i in top_unigram]
count = [i[1] for i in top_unigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#6baed6'] * 20))

p = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Unigrams", tools = "hover", tooltips = "@Word: @counts")
p.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
curdoc().theme = 'dark_minimal'
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.title.align = 'center'
p.xaxis.major_label_orientation = "vertical"
show(p)

In [ ]:
top_bigram = get_top_n_bigram(df_train['excerpt_clean'], 20)
words = [i[0] for i in top_bigram]
count = [i[1] for i in top_bigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#a1dab4'] * 20))

p1 = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Bigrams", tools = "hover", tooltips = "@Word: @counts")
p1.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
# curdoc().theme = 'dark_minimal'
p1.xgrid.grid_line_color = None
p1.title.align = 'center'
p1.y_range.start = 0
p1.xaxis.major_label_orientation = "vertical"
show(p1)

In [ ]:
top_bigram = get_top_n_trigram(df_train['excerpt_clean'], 20)
words = [i[0] for i in top_bigram]
count = [i[1] for i in top_bigram]
source = ColumnDataSource(data = dict(Word = words, counts = count, color = ['#a1dab4'] * 20))

p1 = figure(x_range = words, plot_height = 400, plot_width = 800, title = "Top Bigrams", tools = "hover", tooltips = "@Word: @counts")
p1.vbar(x = 'Word', top = 'counts', width = 0.8, source = source, color = 'color')
# curdoc().theme = 'dark_minimal'
p1.xgrid.grid_line_color = None
p1.title.align = 'center'
p1.y_range.start = 0
p1.xaxis.major_label_orientation = "vertical"
show(p1)

### Stemming

In [ ]:
stemmer = SnowballStemmer(language='english')

tokens = df_train['excerpt'][1].split()
clean_text = ' '

for token in tokens:
    print(token + ' --> ' + stemmer.stem(token))

In [ ]:
def stemWord(text):
    stemmer = SnowballStemmer(language='english')
    tokens = text.split()
    clean_text = ' '
    for token in tokens:
        clean_text = clean_text + " " + stemmer.stem(token)      
    
    return clean_text

print("\033[1mText before stemWord function: \033[0m" + df_train['excerpt'][1])
print("\033[1mText after stemWord function: \033[0m" + stemWord(df_train['excerpt'][1]))

In [ ]:
df_train['excerpt_clean_Stem'] = df_train['excerpt_clean'].apply(stemWord)
df_test['excerpt_clean_Stem'] = df_test['excerpt_clean'].apply(stemWord)

#### Lemmatization

In [ ]:
def lemmatizeWord(text):
    tokens=nlp(text)
    clean_text = ' '
    for token in tokens:
        clean_text = clean_text + " " + token.lemma_      
    
    return clean_text

In [ ]:
df_train['excerpt_clean_Lemm'] = df_train['excerpt_clean'].apply(lemmatizeWord)
df_test['excerpt_clean_Lemm'] = df_test['excerpt_clean'].apply(lemmatizeWord)

In [ ]:
results = Counter()
df_train['excerpt_clean'].str.lower().str.split().apply(results.update)
print(len(results.keys()))

In [ ]:
results = Counter()
df_train['excerpt_clean_Lemm'].str.lower().str.split().apply(results.update)
print(len(results.keys()))

In [ ]:
results = Counter()
df_train['excerpt_clean_Stem'].str.lower().str.split().apply(results.update)
print(len(results.keys()))

# Text Vectorization
* Bag of Words (BoW)
* TD IDF
* Embeddings

## Bag of Words
A bag-of-words is a representation of text that describes the occurrence of words within a document. It involves two things:

A vocabulary of known words.
A measure of the presence of known words. It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.
An n-gram is a contiguous sequence of n items from a given sample of text or speech

An n-gram of size 1 is referred to as a "unigram"; size 2 is a "bigram" (or, less commonly, a "digram"); size 3 is a "trigram".

## Machine Learning
### Unigram

In [ ]:
rmse = lambda y_true, y_pred: np.sqrt(mse(y_true, y_pred))
rmse_loss = lambda Estimator, X, y: rmse(y, Estimator.predict(X))

In [ ]:
# Split into train and test sets


x = df_train['excerpt_clean']
y = df_train['target']

print(len(x), len(y))

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
# Split into train and test sets


x_Lemm = df_train['excerpt_clean_Lemm']
y_Lemm = df_train['target']

print(len(x_Lemm), len(y_Lemm))

x_train_Lemm, x_test_Lemm, y_train_Lemm, y_test_Lemm = train_test_split(x_Lemm, y_Lemm, random_state=42)
print(len(x_train_Lemm), len(y_train_Lemm))
print(len(x_test_Lemm), len(y_test_Lemm))

In [ ]:
# Split into train and test sets


x_Stem = df_train['excerpt_clean_Stem']
y_Stem = df_train['target']

print(len(x_Stem), len(y_Stem))

x_train_Stem, x_test_Stem, y_train_Stem, y_test_Stem = train_test_split(x_Stem, y_Stem, random_state=42)
print(len(x_train_Stem), len(y_train_Stem))
print(len(x_test_Stem), len(y_test_Stem))

In [ ]:
x

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,1)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,1): {val_score}')

In [ ]:
test_predict=model.predict(df_test['excerpt_clean'])

In [ ]:
test_predict

In [ ]:
df_test

In [ ]:
df_submission['target']=test_predict

In [ ]:
df_submission.to_csv('submission.csv',index=False)

### Bigram

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(2,2)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,2): {val_score}')

model_Stem = make_pipeline(
    CountVectorizer(ngram_range=(2,2)),
    LinearRegression(),
)

model_Stem.fit(x_train_Stem,y_train_Stem)

val_score_Stem = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,2): {val_score_Stem}')

model_Lemm = make_pipeline(
    CountVectorizer(ngram_range=(2,2)),
    LinearRegression(),
)

model_Lemm.fit(x_train_Lemm,y_train_Lemm)

val_score_Lemm = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,2): {val_score_Lemm}')

### Trigram

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(3,3)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(3,3): {val_score}')

model_Stem = make_pipeline(
    CountVectorizer(ngram_range=(3,3)),
    LinearRegression(),
)

model_Stem.fit(x_train_Stem,y_train_Stem)

val_score_Stem = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(3,3): {val_score_Stem}')

model_Lemm = make_pipeline(
    CountVectorizer(ngram_range=(3,3)),
    LinearRegression(),
)

model_Lemm.fit(x_train_Lemm,y_train_Lemm)

val_score_Lemm = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(3,3): {val_score_Lemm}')

# Uni + Bi-Gram

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score}')

model_Stem = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    LinearRegression(),
)

model_Stem.fit(x_train_Stem,y_train_Stem)

val_score_Stem = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score_Stem}')

model_Lemm = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    LinearRegression(),
)

model_Lemm.fit(x_train_Lemm,y_train_Lemm)

val_score_Lemm = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score_Lemm}')

### 1+2+3-Gram

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score}')

model_Stem = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    LinearRegression(),
)

model_Stem.fit(x_train_Stem,y_train_Stem)

val_score_Stem = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score_Stem}')

model_Lemm = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    LinearRegression(),
)

model_Lemm.fit(x_train_Lemm,y_train_Lemm)

val_score_Lemm = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score_Lemm}')

### Bi-Gram + Tri-Gram

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    LinearRegression(),
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score}')

model_Stem = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    LinearRegression(),
)

model_Stem.fit(x_train_Stem,y_train_Stem)

val_score_Stem = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score_Stem}')

model_Lemm = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    LinearRegression(),
)

model_Lemm.fit(x_train_Lemm,y_train_Lemm)

val_score_Lemm = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score_Lemm}')

## XGBoost

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,1)),
    xgb.XGBRegressor() ,
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,1): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,1)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Stem,y_train_Stem)

val_score = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,1): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,1)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Lemm,y_train_Lemm)

val_score = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,1): {val_score}')

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    xgb.XGBRegressor() ,
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Stem,y_train_Stem)

val_score = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,2)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Lemm,y_train_Lemm)

val_score = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,2): {val_score}')

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Stem,y_train_Stem)

val_score = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(1,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Lemm,y_train_Lemm)

val_score = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(1,3): {val_score}')

In [ ]:
model = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train,y_train)

val_score = cross_val_score(
    model, 
    x_test, 
    y_test, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Stem,y_train_Stem)

val_score = cross_val_score(
    model, 
    x_test_Stem, 
    y_test_Stem, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score}')

model = make_pipeline(
    CountVectorizer(ngram_range=(2,3)),
    xgb.XGBRegressor() ,
)

model.fit(x_train_Lemm,y_train_Lemm)

val_score = cross_val_score(
    model, 
    x_test_Lemm, 
    y_test_Lemm, 
    scoring=rmse_loss
).mean()

print(f'Validation Score for CountVectorizer(2,3): {val_score}')

In [ ]:
def training_BOW(X_train, y_train, X_test, y_test, model_name, m,n):
    t1 = time.time()
    
    model = make_pipeline(CountVectorizer(ngram_range=(m,n)),model_name)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    MSE = mse(y_test, y_pred)
    rmse =np.sqrt(MSE)
    t2 = time.time()
    training_time = t2-t1 
    
    print("--- Model:", model_name,"---")
    print("MSE: ",MSE)
    print("RMSE: ",rmse)
    print("Training time:",training_time)
    print("\n")

In [ ]:
training_BOW(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),1,2)

In [ ]:
training_BOW(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),1,1)

In [ ]:
training_BOW(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),1,3)

### TF-IDF

TF-IDF (stands for Term-Frequency-Inverse-Document Frequency) weights down the common words occuring in almost all the documents and give more importance to the words that appear in a subset of documents. TF-IDF works by penalising these common words by assigning them lower weights while giving importance to some rare words in a particular document.

In [ ]:
def training_TFIDF(X_train, y_train, X_test, y_test, model_name,ngram_range):
    t1 = time.time()
    
    model = make_pipeline(TfidfVectorizer(binary=True, ngram_range=ngram_range),model_name)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    MSE = mse(y_test, y_pred)
    rmse =np.sqrt(MSE)
    t2 = time.time()
    training_time = t2-t1 
    
    print("--- Model:", model_name,"---")
    print("MSE: ",MSE)
    print("RMSE: ",rmse)
    print("Training time:",training_time)
    print("\n")

In [ ]:
n_gram_dict = { "Unigram" : (1,1), "Unigrams + Bigrams": (1,2), "Bigrams alone": (2,2), "Unigrams + Bigrams + Trigrams": (1,3)}

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),n_gram_dict['Unigram'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,LinearRegression(),n_gram_dict['Unigram'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),n_gram_dict['Unigrams + Bigrams'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,LinearRegression(),n_gram_dict['Unigrams + Bigrams'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),n_gram_dict['Bigrams alone'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,LinearRegression(),n_gram_dict['Bigrams alone'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,xgb.XGBRegressor(),n_gram_dict['Unigrams + Bigrams + Trigrams'])

In [ ]:
training_TFIDF(x_train_Lemm,y_train_Lemm,x_test_Lemm,y_test_Lemm,LinearRegression(),n_gram_dict['Unigrams + Bigrams + Trigrams'])

### Deep Learning
#### Keras

In [ ]:
x_train_Lemm.shape[0]

In [ ]:
from keras.models import Sequential
from keras import layers

#input_dim = x_train_Lemm.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=1, activation='relu'))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', metrics=[RootMeanSquaredError()])
model.summary()

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=26118)
tokenizer.fit_on_texts(df_train['excerpt_clean_Lemm'].to_list())

X_train = tokenizer.texts_to_sequences(df_train['excerpt_clean_Lemm'].to_list())
X_test = tokenizer.texts_to_sequences(df_test['excerpt_clean_Lemm'].to_list())

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(df_train['excerpt_clean_Lemm'][2])
print(X_train[2])

In [ ]:
vocab_size

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 800

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', metrics=[RootMeanSquaredError()])
model.summary()

In [ ]:
training_labels_final = np.array(df_train.target)

In [ ]:
history = model.fit(X_train, training_labels_final,
                    epochs=20,
                    validation_split=0.1,
                    batch_size=32)

### Glove Embeddings

What if, instead of learning the embeddings for yourself, you could instead use prelearned embeddings, where researchers have already done the hard work of turning words into vectors and those vectors are proven? One example of this is the GloVe (Global Vectors for WordRepresentation) model developed by Jeffrey Pennington, Richard Socher, and Christopher Manning at Stanford

Pretrained Word Embeddings are the embeddings learned in one task that are used for solving another similar task. These embeddings are trained on large datasets, saved, and then used for solving other tasks. That’s why pretrained word embeddings are a form of Transfer Learning.

In [ ]:
#text = df_train.excerpt
text = df_train.excerpt_clean_Lemm

In [ ]:
vocab_size = 26118
embedding_dim = 512
max_length = 500
trunc_type='post'
pad_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index

In [ ]:
X_train_Seq = tokenizer.texts_to_sequences(text)
X_train = pad_sequences(X_train_Seq,maxlen=max_length, 
                                truncating=trunc_type, padding=pad_type)


In [ ]:
glove_embeddings = dict()
f = open('../input/glove-global-vectors-for-word-representation/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings[word] = coefs
f.close()

In [ ]:
glove_embeddings['study']

In [ ]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(glove_embeddings[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(50)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
embedding_dim = 50
vocab_size = 26118

In [ ]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index > vocab_size - 1:
         break
    else:
        embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
model = tf.keras.Sequential([
 tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim,
 return_sequences=True)),
 tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
 tf.keras.layers.Dense(24, activation='relu'),
 tf.keras.layers.Dense(1)
])

In [ ]:
learning_rate = 0.00003
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(learning_rate), metrics=[RootMeanSquaredError()])
model.summary()

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_root_mean_squared_error', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
num_epochs = 100
history = model.fit(X_train, training_labels_final, epochs=num_epochs, 
                    validation_split=0.3,
                    batch_size=32,callbacks=[early_stopping,learning_rate_reduction])

In [ ]:
padding_type='post'
sample_sequences = tokenizer.texts_to_sequences(df_test['excerpt'])
excerpt_padded = pad_sequences(sample_sequences, padding=padding_type, 
                             maxlen=max_length) 
values = model.predict(excerpt_padded)
submission_file= "/kaggle/input/commonlitreadabilityprize/sample_submission.csv"
sample_submission = pd.read_csv(submission_file)
sample_submission["target"] = values
sample_submission.to_csv("submission.csv", index=False)